<a href="https://colab.research.google.com/github/OlegMityaev/iVision/blob/main/iVision404.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.metrics import MeanSquaredError
from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
all_us = pd.read_csv('/content/drive/MyDrive/iVision404/01_all_users.csv')
ev_log = pd.read_csv('/content/drive/MyDrive/iVision404/02_events_log.csv')
lk_ev = pd.read_csv('/content/drive/MyDrive/iVision404/03_lk_events_log.csv')
block = pd.read_csv('/content/drive/MyDrive/iVision404/04_is_blocked.csv')

Удаляю столбец даты события

In [46]:
ev_log.drop(columns='event_date', axis = 0, inplace = True)
ev_log

,contract_id,event_type
0,29516,Добавление в Обращались с номеров
1,29516,Обращение в службу заботы о клиентах
2,29516,Выключение IPTV-пакета
3,29516,Включение IPTV-пакета
4,29516,"Включение услуги ""Ограничение функций ТВ"""
...,...,...
7189,47613,Обращение в службу заботы о клиентах
7190,47613,Обращение в службу заботы о клиентах
7191,47613,Обращение в службу заботы о клиентах
7192,5023,Манипуляции с оборудованием


Считаю количество событий, совершенных одним айди, складываю в строку с этим айди, лишнее удаляю

In [47]:
len(ev_log)

7194

In [48]:
count = 0
coun1 = 0
for i in range(0, 7194):
    count = 0
    print(coun1)
    if i + coun1 == 7194:
        break
    for j in range(i + 1, 7194):
        print(i + coun1, j + coun1)
        if j + coun1 == 7194:
            break
        if ev_log.at[i + coun1, 'contract_id'] == ev_log.at[j + coun1, 'contract_id']:
            count += 1
            ev_log.at[i + coun1, 'event_type'] = count
            ev_log.drop(labels=j + coun1, axis=0, inplace=True)
        elif ev_log.at[i + coun1, 'contract_id'] != ev_log.at[j + coun1, 'contract_id']:
            ev_log.at[i + coun1, 'event_type'] = count
            coun1 += count
            print(count)
            break
        
ev_log

Выходные данные были обрезаны до нескольких последних строк (5000).
3988 3996
3988 3997
3988 3998
3988 3999
10
3059
3999 4000
0
3059
4000 4001
0
3059
4001 4002
4001 4003
4001 4004
4001 4005
4001 4006
4
3063
4006 4007
4006 4008
4006 4009
4006 4010
4006 4011
4006 4012
4006 4013
4006 4014
7
3070
4014 4015
4014 4016
1
3071
4016 4017
4016 4018
4016 4019
2
3073
4019 4020
0
3073
4020 4021
4020 4022
4020 4023
2
3075
4023 4024
4023 4025
4023 4026
4023 4027
4023 4028
4023 4029
4023 4030
4023 4031
4023 4032
4023 4033
9
3084
4033 4034
4033 4035
4033 4036
2
3086
4036 4037
4036 4038
4036 4039
4036 4040
4036 4041
4036 4042
4036 4043
4036 4044
7
3093
4044 4045
4044 4046
4044 4047
2
3095
4047 4048
0
3095
4048 4049
4048 4050
4048 4051
2
3097
4051 4052
4051 4053
4051 4054
4051 4055
3
3100
4055 4056
4055 4057
1
3101
4057 4058
4057 4059
4057 4060
2
3103
4060 4061
0
3103
4061 4062
4061 4063
4061 4064
4061 4065
3
3106
4065 4066
4065 4067
1
3107
4067 4068
4067 4069
1
3108
4069 4070
4069 4071
4069 4072
4069 40

,contract_id,event_type
0,29516,14
15,27462,5
21,4869,1
23,33846,2
26,30031,0
...,...,...
7182,47571,1
7184,47584,0
7185,46104,1
7187,47613,4


In [60]:

ev_log.to_csv('/content/drive/MyDrive/iVision404/ev_log_num.csv')

In [50]:
ev_log.head(20)

,contract_id,event_type
0,29516,14
15,27462,5
21,4869,1
23,33846,2
26,30031,0
27,34201,0
28,31953,1
30,6799,2
33,34348,2
36,40463,1


In [51]:
lk_ev = pd.read_csv('/content/drive/MyDrive/iVision404/03_lk_events_log.csv')
lk_ev

,contract_id,event_date,name
0,33846,2021-03-19 15:12:52,Информер ВК. Показ
1,33846,2021-03-27 18:46:04,Информер ВК. Показ
2,33846,2021-04-03 18:55:22,Информер ВК. Показ
3,22754,2021-03-09 19:40:53,Включение интернета на 20 минут
4,22754,2021-03-09 19:40:53,Включение интернета на 20 минут
...,...,...,...
19232,5023,2021-04-29 7:50:44,Информер ВК. Показ
19233,47627,2021-03-17 10:38:42,Информер ВК. Показ
19234,47627,2021-03-23 8:21:29,Информер ВК. Показ
19235,47627,2021-04-09 13:04:26,Информер ВК. Показ


In [52]:
lk_ev.drop(columns='event_date', axis = 0, inplace = True)
lk_ev

,contract_id,name
0,33846,Информер ВК. Показ
1,33846,Информер ВК. Показ
2,33846,Информер ВК. Показ
3,22754,Включение интернета на 20 минут
4,22754,Включение интернета на 20 минут
...,...,...
19232,5023,Информер ВК. Показ
19233,47627,Информер ВК. Показ
19234,47627,Информер ВК. Показ
19235,47627,Информер ВК. Показ


In [53]:
len(lk_ev)

19237

In [54]:
count = 0
coun1 = 0
for i in range(0, 19237):
    count = 0
    print(coun1)
    if i + coun1 == 19234:
        break
    for j in range(i + 1, 19237):
        print(i + coun1, j + coun1)
        if j + coun1 == 19237:
            break
        if lk_ev.at[i + coun1, 'contract_id'] == lk_ev.at[j + coun1, 'contract_id']:
            count += 1
            lk_ev.at[i + coun1, 'name'] = count
            lk_ev.drop(labels=j + coun1, axis=0, inplace=True)
        elif lk_ev.at[i + coun1, 'contract_id'] != lk_ev.at[j + coun1, 'contract_id']:
            lk_ev.at[i + coun1, 'name'] = count
            coun1 += count
            print(count)
            break
        
lk_ev

Выходные данные были обрезаны до нескольких последних строк (5000).
15461 15469
15461 15470
15461 15471
15461 15472
15461 15473
15461 15474
15461 15475
15461 15476
15461 15477
15461 15478
15461 15479
15461 15480
15461 15481
15461 15482
15461 15483
15461 15484
15461 15485
15461 15486
15461 15487
15461 15488
15461 15489
27
12381
15489 15490
15489 15491
15489 15492
15489 15493
15489 15494
4
12385
15494 15495
15494 15496
15494 15497
15494 15498
15494 15499
15494 15500
5
12390
15500 15501
15500 15502
15500 15503
15500 15504
15500 15505
4
12394
15505 15506
15505 15507
1
12395
15507 15508
15507 15509
15507 15510
15507 15511
15507 15512
15507 15513
15507 15514
15507 15515
15507 15516
8
12403
15516 15517
15516 15518
15516 15519
15516 15520
15516 15521
15516 15522
15516 15523
15516 15524
7
12410
15524 15525
15524 15526
15524 15527
15524 15528
15524 15529
15524 15530
15524 15531
15524 15532
15524 15533
15524 15534
9
12419
15534 15535
15534 15536
15534 15537
2
12421
15537 15538
15537 15539
15537 1

,contract_id,name
0,33846,2
3,22754,1
5,36009,1
7,23100,2
10,25266,4
...,...,...
19192,46104,5
19198,26721,4
19203,47613,17
19221,5023,11


In [55]:
lk_ev.to_csv('/content/drive/MyDrive/iVision404/lk_ev_num.csv')


In [61]:
ev_log_num = pd.read_csv('/content/drive/MyDrive/iVision404/ev_log_num.csv')
ev_log_num.drop(columns='Unnamed: 0', axis=0, inplace=True)
ev_log_num

,contract_id,event_type
0,29516,14
1,27462,5
2,4869,1
3,33846,2
4,30031,0
...,...,...
1834,47571,1
1835,47584,0
1836,46104,1
1837,47613,4


In [62]:
lk_ev_num = pd.read_csv('/content/drive/MyDrive/iVision404/lk_ev_num.csv')
lk_ev_num.drop(columns='Unnamed: 0', axis=0, inplace=True)
lk_ev_num

,contract_id,name
0,33846,2
1,22754,1
2,36009,1
3,23100,2
4,25266,4
...,...,...
3718,46104,5
3719,26721,4
3720,47613,17
3721,5023,11


In [63]:
all_us = pd.read_csv('/content/drive/MyDrive/iVision404/01_all_users.csv')
all_us

,contract_id,day_or_month_contract
0,29516,0
1,27462,0
2,21218,0
3,4869,0
4,33846,0
...,...,...
6800,4859,0
6801,47612,1
6802,47613,1
6803,5023,0


Удаляю айди, для которых нет данных

In [65]:
df = pd.DataFrame({})
df.insert(0, 'contract_id', 0)

for i in range(len(block)):
    for j in range(len(all_us)):
        print(i, j)
        if block.at[i, 'contract_id'] == all_us.at[j, 'contract_id']:
            df.at[i, 'contract_id'] = all_us.at[j, 'contract_id']
            print('ADD')
            break

df.to_csv('contract_id.csv')

Выходные данные были обрезаны до нескольких последних строк (5000).
1801 597
1801 598
1801 599
1801 600
1801 601
1801 602
1801 603
1801 604
1801 605
1801 606
1801 607
1801 608
1801 609
1801 610
1801 611
1801 612
1801 613
1801 614
1801 615
1801 616
1801 617
1801 618
1801 619
1801 620
1801 621
1801 622
1801 623
1801 624
1801 625
1801 626
1801 627
1801 628
1801 629
1801 630
1801 631
1801 632
1801 633
1801 634
1801 635
1801 636
1801 637
1801 638
1801 639
1801 640
1801 641
1801 642
1801 643
1801 644
1801 645
1801 646
1801 647
1801 648
1801 649
1801 650
1801 651
1801 652
1801 653
1801 654
1801 655
1801 656
1801 657
1801 658
1801 659
1801 660
1801 661
1801 662
1801 663
1801 664
1801 665
1801 666
1801 667
1801 668
1801 669
1801 670
1801 671
1801 672
1801 673
1801 674
1801 675
1801 676
1801 677
1801 678
1801 679
1801 680
1801 681
1801 682
1801 683
1801 684
1801 685
1801 686
1801 687
1801 688
1801 689
1801 690
1801 691
1801 692
1801 693
1801 694
1801 695
1801 696
1801 697
1801 698
1801 699
1801 

KeyboardInterrupt: ignored

In [42]:
id = pd.read_csv('/content/drive/MyDrive/iVision404/contract_id.csv')

Собираю все данные в одну таблицу в числовом виде

In [ ]:
id.insert(0, 'day_or_month_contract', 0)
id.insert(1, 'event_type', 0)
id.insert(2, 'name', 0)
id.insert(3, 'blocked', 0)

for i in range(len(id)):
    for j in range(len(ev_log)):
        print(i, j)
        if id.at[i, 'contract_id'] == all_us.at[j, 'contract_id']:
            id.at[i, "day_or_month_contract"] = all_us.at[j, 'day_or_month_contract']
            print('ADD')
        if id.at[i, 'contract_id'] == ev_log_num.at[j, 'contract_id']:
            id.at[i, "event_type"] = ev_log_num.at[j, 'event_type']
            print('ADD')
        if id.at[i, 'contract_id'] == lk_ev_num.at[j, 'contract_id']:
            id.at[i, 'name'] = lk_ev_num.at[j, 'name']
            print('ADD')
        if id.at[i, 'contract_id'] == block.at[j, 'contract_id']:
            id.at[i, 'blocked'] = block.at[j, 'blocked']
            print('ADD')
            break

id.to_csv('x.csv')

Выходные данные были обрезаны до нескольких последних строк (5000).
994 57
994 58
994 59
994 60
994 61
994 62
994 63
994 64
994 65
994 66
994 67
994 68
994 69
994 70
994 71
994 72
994 73
994 74
994 75
994 76
994 77
994 78
994 79
994 80
994 81
994 82
994 83
994 84
994 85
994 86
994 87
994 88
994 89
994 90
994 91
994 92
994 93
994 94
994 95
994 96
994 97
994 98
994 99
994 100
994 101
994 102
994 103
994 104
994 105
994 106
994 107
994 108
994 109
994 110
994 111
994 112
994 113
994 114
994 115
994 116
994 117
994 118
994 119
994 120
994 121
994 122
994 123
994 124
994 125
994 126
994 127
994 128
994 129
994 130
994 131
994 132
994 133
994 134
994 135
994 136
994 137
994 138
994 139
994 140
994 141
994 142
994 143
994 144
994 145
994 146
994 147
994 148
994 149
994 150
994 151
994 152
994 153
994 154
994 155
994 156
994 157
994 158
994 159
994 160
994 161
994 162
994 163
994 164
994 165
994 166
994 167
994 168
994 169
994 170
994 171
994 172
994 173
994 174
994 175
994 176
994 177
994 178

KeyboardInterrupt: ignored

Убираю ненужные для нейросети данные

In [77]:
df_x = pd.read_csv('/content/drive/MyDrive/iVision404/x.csv')
df_x.drop(columns='contract_id', axis=0, inplace=True)
df_x

,Unnamed: 0,day_or_month_contract,event_type,name,blocked
0,0,0,14,0,1
1,1,0,3,0,1
2,2,0,0,0,1
3,3,0,1,0,1
4,4,0,9,2,1
...,...,...,...,...,...
6499,6499,0,0,0,0
6500,6500,0,0,0,0
6501,6501,0,4,0,0
6502,6502,0,2,0,0


In [67]:
df_y = pd.DataFrame(df_x['blocked'])
df_y

,blocked
0,1
1,1
2,1
3,1
4,1
...,...
6499,0
6500,0
6501,0
6502,0


In [78]:
df_x.drop(columns='blocked', axis=0, inplace=True)
df_x.drop(columns='Unnamed: 0', axis=0, inplace=True)

Привожу данные к одному масштабу

In [79]:
scaler = StandardScaler()
scaler.fit(df_x)
x = scaler.transform(df_x)
x

array([[-0.22257605,  4.55075222, -0.22629364],
       [-0.22257605,  0.76626987, -0.22629364],
       [-0.22257605, -0.26586168, -0.22629364],
       ...,
       [-0.22257605,  1.11031372, -0.22629364],
       [-0.22257605,  0.42222602, -0.22629364],
       [-0.22257605, -0.26586168, -0.22629364]])

In [89]:
y = to_categorical(df_y)
y

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

Сама нейросеть

In [90]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [92]:
model = Sequential()
model.add(Dense(20, input_dim = 3, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=('accuracy'))

In [93]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 20)                80        
                                                                 
 dense_13 (Dense)            (None, 10)                210       
                                                                 
 dense_14 (Dense)            (None, 10)                110       
                                                                 
 dense_15 (Dense)            (None, 2)                 22        
                                                                 
Total params: 422
Trainable params: 422
Non-trainable params: 0
_________________________________________________________________


In [94]:
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
163/163 [==============================] - 1s 2ms/step - loss: 0.3885 - accuracy: 0.9266
Epoch 2/20
163/163 [==============================] - 0s 2ms/step - loss: 0.2162 - accuracy: 0.9389
Epoch 3/20
163/163 [==============================] - 0s 2ms/step - loss: 0.2120 - accuracy: 0.9395
Epoch 4/20
163/163 [==============================] - 0s 2ms/step - loss: 0.2104 - accuracy: 0.9398
Epoch 5/20
163/163 [==============================] - 0s 2ms/step - loss: 0.2094 - accuracy: 0.9398
Epoch 6/20
163/163 [==============================] - 0s 2ms/step - loss: 0.2087 - accuracy: 0.9398
Epoch 7/20
163/163 [==============================] - 0s 2ms/step - loss: 0.2087 - accuracy: 0.9400
Epoch 8/20
163/163 [==============================] - 0s 2ms/step - loss: 0.2082 - accuracy: 0.9400
Epoch 9/20
163/163 [==============================] - 0s 2ms/step - loss: 0.2084 - accuracy: 0.9400
Epoch 10/20
163/163 [==============================] - 0s 2ms/step - loss: 0.2086 - accuracy: 0.9400

In [109]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [95]:
test = pd.read_csv('/content/drive/MyDrive/iVision404/test.csv')
test

,contract_id,blocked
0,1034,NaN
1,1074,NaN
2,1145,NaN
3,1261,NaN
4,1315,NaN
...,...,...
296,274468,NaN
297,274482,NaN
298,274486,NaN
299,274540,NaN


In [ ]:
ytest = test.copy()
ytest.drop(columns='blocked', axis=0, inplace=True)
ytest.insert(1, 'day_or_month_contract', 0)
ytest.insert(2, 'event_type', 0)
ytest.insert(3, 'name', 0)

for i in range(len(ytest)):
    for j in range(len(ev_log_num)):
        print(i, j)
        if ytest.at[i, 'contract_id'] == all_us.at[j, 'contract_id']:
            ytest.at[i, "day_or_month_contract"] = all_us.at[j, 'day_or_month_contract']
            print('ADD')
        if ytest.at[i, 'contract_id'] == ev_log_num.at[j, 'contract_id']:
            ytest.at[i, "event_type"] = ev_log_num.at[j, 'event_type']
            print('ADD')
        if ytest.at[i, 'contract_id'] == lk_ev_num.at[j, 'contract_id']:
            ytest.at[i, 'name'] = lk_ev_num.at[j, 'name']
            print('ADD')
            break

print(ytest)
ytest.to_csv('/content/drive/MyDrive/iVision404/ytest.csv')

In [108]:
ytest = pd.read_csv('/content/drive/MyDrive/iVision404/ytest.csv')
# ytest.drop(columns='contract_id', axis=0, inplace=True)
ytest.drop(columns='Unnamed: 0', axis=0, inplace=True)
ytest

,contract_id,day_or_month_contract,event_type,name
0,1034,0,1,10
1,1074,0,0,0
2,1145,0,0,2
3,1261,0,0,5
4,1315,0,0,3
...,...,...,...,...
296,274468,0,5,2
297,274482,0,1,2
298,274486,0,2,2
299,274540,0,0,10


In [98]:
testy = scaler.transform(ytest)
testy

array([[-2.22576055e-01,  7.81821664e-02,  1.98862564e+00],
       [-2.22576055e-01, -2.65861684e-01, -2.26293644e-01],
       [-2.22576055e-01, -2.65861684e-01,  2.16690212e-01],
       [-2.22576055e-01, -2.65861684e-01,  8.81165996e-01],
       [-2.22576055e-01, -2.65861684e-01,  4.38182140e-01],
       [-2.22576055e-01, -2.65861684e-01, -2.26293644e-01],
       [-2.22576055e-01, -2.65861684e-01,  1.98862564e+00],
       [-2.22576055e-01, -2.65861684e-01, -2.26293644e-01],
       [-2.22576055e-01, -2.65861684e-01, -2.26293644e-01],
       [-2.22576055e-01,  7.81821664e-02, -2.26293644e-01],
       [-2.22576055e-01, -2.65861684e-01, -2.26293644e-01],
       [-2.22576055e-01, -2.65861684e-01, -2.26293644e-01],
       [-2.22576055e-01, -2.65861684e-01, -2.26293644e-01],
       [-2.22576055e-01,  7.81821664e-02,  5.53249648e+00],
       [-2.22576055e-01, -2.65861684e-01, -2.26293644e-01],
       [-2.22576055e-01, -2.65861684e-01, -2.26293644e-01],
       [-2.22576055e-01, -2.65861684e-01

In [99]:
pred = model.predict(x_test)
pred

array([[0.96872264, 0.03127741],
       [0.96872264, 0.03127741],
       [0.96872264, 0.03127741],
       ...,
       [0.816417  , 0.18358293],
       [0.96872264, 0.03127741],
       [0.9076434 , 0.09235667]], dtype=float32)

Предположение

In [100]:
predy = model.predict(testy)
predy

array([[8.27582359e-01, 1.72417641e-01],
       [9.68722641e-01, 3.12774070e-02],
       [8.48686993e-01, 1.51313037e-01],
       [8.44854295e-01, 1.55145749e-01],
       [8.40785027e-01, 1.59215018e-01],
       [9.68722641e-01, 3.12774070e-02],
       [8.45766127e-01, 1.54233813e-01],
       [9.68722641e-01, 3.12774070e-02],
       [9.68722641e-01, 3.12774070e-02],
       [9.45158660e-01, 5.48413917e-02],
       [9.68722641e-01, 3.12774070e-02],
       [9.68722641e-01, 3.12774070e-02],
       [9.68722641e-01, 3.12774070e-02],
       [7.65476465e-01, 2.34523505e-01],
       [9.68722641e-01, 3.12774070e-02],
       [9.68722641e-01, 3.12774070e-02],
       [9.22391593e-01, 7.76084438e-02],
       [9.68722641e-01, 3.12774070e-02],
       [9.68722641e-01, 3.12774070e-02],
       [8.54484737e-01, 1.45515323e-01],
       [9.68722641e-01, 3.12774070e-02],
       [9.68722641e-01, 3.12774070e-02],
       [9.68722641e-01, 3.12774070e-02],
       [9.22391593e-01, 7.76084438e-02],
       [8.427961

Точность

In [101]:
p = []
a = []
for i in range(len(pred)):
    if pred[i][0] > pred[i][1]:
        p.append(0)
    else:
        p.append(1)
    if p[i] == df_y['blocked'][i]:
        a.append(1)
    else:
        a.append(0)
print('Точность:')
print(sum(a)/len(a))

Точность:
0.4711760184473482


In [102]:
result = pd.DataFrame(list(predy))
result.to_csv('/content/drive/MyDrive/iVision404/result.csv')

In [103]:
result

,0,1
0,0.827582,0.172418
1,0.968723,0.031277
2,0.848687,0.151313
3,0.844854,0.155146
4,0.840785,0.159215
...,...,...
296,0.787702,0.212298
297,0.803729,0.196271
298,0.781362,0.218638
299,0.845766,0.154234


In [105]:
result1 = pd.DataFrame()
result1.insert(0, 'blocked', 0)
result1
for i in range(len(result)):
    if result.at[i, 0] > result.at[i, 1]:
        result1.at[i, 'blocked'] = 0
    else:
        result1.at[i, 'blocked'] = 1
result1.to_csv('/content/drive/MyDrive/iVision404/res1.csv')
result1.head(50)

,blocked
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [106]:
test.drop(columns='blocked', axis=0, inplace=True)

Решение

In [107]:
itog = pd.concat([
    test,
    result1], axis=1, join='inner'
)
itog.to_csv('/content/drive/MyDrive/iVision404/itog.csv')